# Creating TFRecords from CSV created by https://www.robots.ox.ac.uk/~vgg/software/via/via_demo.html

This script is designed to convert annotations from a CSV file (formatted by VGG Image Annotator) into TensorFlow's TFRecord format.

### Input CSV File Format

The CSV file should have the following columns:

    filename: Name of the image file.
    region_shape_attributes: Bounding box details (e.g., x, y, width, height).
    region_attributes: Additional information about each region (e.g., object class).

### Expected Directory Structure

    data/: Directory containing the image files referenced in the CSV file.
    CSV file: The annotation file created by VGG Image Annotator.

### Script Functionality

    Read CSV File: Parses the CSV file to extract bounding box and class information.
    Filter Annotations: Focuses on the specified object classes (person, bicycle, car).
    Read and Encode Images: Reads images from the data/ directory and encodes them.
    Convert to TFRecord: Converts the data to TFRecord format and writes to a file.

### How to Use

    Place the CSV file and the data/ directory in the same directory as the script.
    Update the script with the path to your CSV file and output TFRecord file.
    Run the script. This will generate a TFRecord file in the specified output location.
    This file can then be used as dataset for yolov3-tf2 training (training_yolov2-tf2.ipynb)

In [ ]:
# Change X to the GPU number you want to use,
# otherwise you will get a Python error
# e.g. USE_GPU = 4
USE_GPU = 4

In [ ]:
# Import TensorFlow 
import tensorflow as tf

# Print the installed TensorFlow version
print(f'TensorFlow version: {tf.__version__}\n')

# Get all GPU devices on this server
gpu_devices = tf.config.list_physical_devices('GPU')

# Print the name and the type of all GPU devices
print('Available GPU Devices:')
for gpu in gpu_devices:
    print(' ', gpu.name, gpu.device_type)
    
# Set only the GPU specified as USE_GPU to be visible
tf.config.set_visible_devices(gpu_devices[USE_GPU], 'GPU')

# Get all visible GPU  devices on this server
visible_devices = tf.config.get_visible_devices('GPU')

# Print the name and the type of all visible GPU devices
print('\nVisible GPU Devices:')
for gpu in visible_devices:
    print(' ', gpu.name, gpu.device_type)
    
# Set the visible device(s) to not allocate all available memory at once,
# but rather let the memory grow whenever needed
for gpu in visible_devices:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
import os
import json
import tensorflow as tf
import pandas as pd
from PIL import Image
from io import BytesIO
from collections import namedtuple

def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def float_list_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def bytes_list_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def int64_list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def class_text_to_int(row_label):
    if row_label == 'person':
        return 1
    elif row_label == 'bicycle':
        return 2
    elif row_label == 'car':
        return 3
    else:
        None
        
def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_img = fid.read()
    
    encoded_img_io = BytesIO(encoded_img)
    image = Image.open(encoded_img_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'png'  # or b'jpeg' if your images are in jpeg format

    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

        x = row['x']
        y = row['y']
        w = row['width']
        h = row['height']

        xmin = x / width
        xmax = (x + w) / width
        ymin = y / height
        ymax = (y + h) / height

        xmins.append(xmin)
        xmaxs.append(xmax)
        ymins.append(ymin)
        ymaxs.append(ymax)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': int64_feature(height),
        'image/width': int64_feature(width),
        'image/filename': bytes_feature(filename),
        'image/source_id': bytes_feature(filename),
        'image/encoded': bytes_feature(encoded_img),
        'image/format': bytes_feature(image_format),
        'image/object/bbox/xmin': float_list_feature(xmins),
        'image/object/bbox/xmax': float_list_feature(xmaxs),
        'image/object/bbox/ymin': float_list_feature(ymins),
        'image/object/bbox/ymax': float_list_feature(ymaxs),
        'image/object/class/text': bytes_list_feature(classes_text),
        'image/object/class/label': int64_list_feature(classes),
    }))
    return tf_example



# Assuming you have the following helper function
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

# Read the CSV file
annotations_df = pd.read_csv('via_project_15Jan2024_10h57m_csv.csv')
annotations_df['region_attributes'] = annotations_df['region_attributes'].apply(json.loads)
annotations_df['class'] = annotations_df['region_attributes'].apply(lambda x: x['type'])
annotations_df['x'] = annotations_df['region_shape_attributes'].apply(lambda x: json.loads(x)['x'])
annotations_df['y'] = annotations_df['region_shape_attributes'].apply(lambda x: json.loads(x)['y'])
annotations_df['width'] = annotations_df['region_shape_attributes'].apply(lambda x: json.loads(x)['width'])
annotations_df['height'] = annotations_df['region_shape_attributes'].apply(lambda x: json.loads(x)['height'])

# Filter out unwanted classes
annotations_df = annotations_df[annotations_df['class'].isin(['person', 'bicycle', 'car'])]

output_path = 'train.record'
writer = tf.io.TFRecordWriter(output_path)
path = os.path.join(os.getcwd(), 'data')

grouped = split(annotations_df, 'filename')
for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())

writer.close()
output_path = os.path.join(os.getcwd(), output_path)
print('Successfully created the TFRecord file:', output_path)
